In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
matplotlib.use('agg')
%matplotlib inline


In [ ]:
df_orig = pd.read_csv('data/priority_places_for_food_oct22.csv')
df_reweighted = pd.read_csv('data/priority_places_london_weighting.csv')

lsoa_lad_lookup = pd.read_csv('data/LSOA_(2011)_to_LSOA_(2021)_to_Local_Authority_District_(2022)_Lookup_for_England_and_Wales.csv')
gdf = gpd.read_file('data/Lower_Layer_Super_Output_Areas_(December_2011)_Boundaries_Full_Extent_(BFE)_EW_V3.zip')

# Load consituency centroid data - used for labels
constituency = pd.read_csv('data/Westminster_Parliamentary_Constituencies_(Dec_2021)_UK_BFC.csv')
constituency = gpd.GeoDataFrame(constituency, geometry=gpd.points_from_xy(constituency['LONG'], constituency['LAT']), crs=4326)

birmingham_pantries = pd.read_csv('data/birmingham_pantries_cafes.csv')
postcodes = pd.read_csv('data/ONS_Postcode_Directory_(Latest)_Centroids.csv', usecols=['X', 'Y', 'pcd', 'pcd2', 'pcds', 'lsoa11', 'lat', 'long', 'oslaua'])
postcodes = postcodes[postcodes['oslaua']=='E08000025']
birmingham_pantries = birmingham_pantries.merge(postcodes, how='left', left_on='pcd', right_on='pcds', indicator=True)
# One of the postcodes - B62 8HX - falls outside the Birmingham Local Authority boundary
birmingham_pantries = birmingham_pantries[birmingham_pantries._merge=='both']
birmingham_pantries = gpd.GeoDataFrame(birmingham_pantries, geometry=gpd.points_from_xy(birmingham_pantries['X'],birmingham_pantries['Y']), crs=27700)

In [ ]:
def lad_level_choropleth_for_ppfi(df, lad_name, lsoa_lad_lookup, gdf, constituency,title, additional_data_points=None, column='pp_dec_combined'):

    pp_lad = df[df['geo_code'].isin(lsoa_lad_lookup[lsoa_lad_lookup['LAD22NM'].isin(lad_name)]['F_LSOA11CD'].values)]
    gdf = gdf[gdf['LSOA11CD'].isin(pp_lad.geo_code.values)]
    gdf = gdf.merge(pp_lad, left_on='LSOA11CD', right_on='geo_code')

    fig, ax = plt.subplots(1, 1)
    fig.suptitle("Priority Places for Food Index", fontsize=12, y=0.96, x=0.31)
    ax.set_title("%s 2011 Lower Super Output Areas" % title, fontsize=8, loc='left')
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="4%", pad=0.2)

    ax = gdf.plot(column=column, legend=True, cmap='OrRd_r', ax=ax, cax=cax)
    ax.axis('off')


    cax.set_yticks(range(1,11))
    cax.set_yticklabels(['1 Highest Priority','2','3','4','5','6','7','8','9','10 Lowest Priority'])   
    cax.tick_params(axis='y', which='major', labelsize=8)
    cax.invert_yaxis()

    lad_constituency = constituency.to_crs(27700).overlay(gdf, how='intersection')
    lad_constituency['name'] = lad_constituency['PCON21NM']#.str.replace('Birmingham, ', '')               

    if additional_data_points is not None:
        additional_data_points.plot(ax=ax, color='black')
    #else:
    #    for i, row in lad_constituency.iterrows(): 
            #ax.annotate(row['name'], xy=(row.geometry.x,row.geometry.y), color='w', fontsize=6, ha='center', weight='bold')
            #ax.annotate(row['name'], xy=(row.geometry.x,row.geometry.y), color='k', fontsize=5, ha='center', weight='bold')

    ax.annotate("""
        The Priority Places for Food index has been developed by the 
        Consumer Data Research Centre at the University of Leeds in 
        collaboration with Which?. See https://data.cdrc.ac.uk/dataset/
        priority-places-food-index/ for more information.
        """, 
        xy=(0.0,-0.1), xycoords='axes fraction', fontsize=6)

    plt.savefig('data/PPFI-%s.png' % title, dpi=300, bbox_inches='tight')

In [ ]:
london = [    
    'City of London',
    'Westminster',
    'Kensington and Chelsea',
    'Hammersmith and Fulham',
    'Wandsworth',
    'Lambeth',
    'Southwark',
    'Tower Hamlets',
    'Hackney',
    'Islington',
    'Camden',
    'Brent',
    'Ealing',
    'Hounslow',
    'Richmond upon Thames',
    'Kingston upon Thames',
    'Merton',
    'Sutton',
    'Croydon',
    'Bromley',
    'Lewisham',
    'Greenwich',
    'Bexley',
    'Havering',
    'Barking and Dagenham',
    'Redbridge',
    'Newham',
    'Waltham Forest',
    'Haringey',
    'Enfield',
    'Barnet',
    'Harrow',
    'Hillingdon']

lad_level_choropleth_for_ppfi(df=df_reweighted,
                              lad_name=london, 
                              lsoa_lad_lookup=lsoa_lad_lookup, 
                              gdf=gdf, 
                              constituency=constituency, 
                              title='New Weighting London', 
                              column='combined_decile')